# Basketball analytics: distilling and summarizing Information

#### Due: May 11 at 10 pm

When analyzing data, setting a goal is often helpful. In this assignment, the focus is on understanding how NMF behaves, and further analyzing player data.

In all the problems below, take a step back and think about each procedure as a piece in a bigger puzzle of understanding the game of basketball and its players. This goal should guide any decisions we make, and insights we interpret.

## Preparing Data

In the previous notebook `07-Shooting-Pattern-Analysis`, we computed smoothed shot patterns for 362 players that played during 2016-17 regular season. Save the matrix `X` from Non-negative matrix factorization (NMF) section.

Please create this file from saving the appropriate variable into a picke file called `allpatterns2016-17.pkl`. After saving the file, you can load it via the following command:

In [ ]:
import pickle
X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))


## Non-negative Matrix Factorization (NMF) notation

Non-negative matrix factorization was used on the smoothed shooting pattern data of around 360 players. The result was useful in
* Bases: Identifying modes of shooting style (number of modes was determined by `n_components` argument to `NMF` function)
* Coefficients: How each players shooting style could be expressed as a linear combination of these bases (matrix multiplication between the bases and coefficients achieve this)

Recall the following. Given some matrix $X$ is $p\times n$ matrix, NMF computes the following factorization:
$$ \min_{W,H} \| X - WH \|_F\\
\text{ subject to } W\geq 0,\ H\geq 0, $$
where $W$ is ${p\times r}$ matrix and $H$ is ${r\times n}$ matrix.


## Problem 1

__PSTAT 134 and 234__: Experiment with different number of `n_components` to change the number of bases vectors. Visualize the bases vectors.

What value of $r$ seem to be too small? (`r` is too small to represent diversity of shooting modes) What value of $r$ seem to be too large? (`r` is too large and some bases seem to be duplicated). Note that, if a basis were a perfect duplicate of another (they will not be, but could be similar), you would use one basis instead of two.

__PSTAT 234 (optional for 134)__: Choose two different choices for number of components, say $r_1=3$ and $r_2=20$. Reconstruct the shooting pattern of at least two player using 3 bases and 20 bases. Is there any difference between the reconstruction?

- For a given player, plot the original shooting frequencies and corresponding reconstruction for $r \in \{3,20\}$.

Compute the difference: i.e., the norm of the difference  $ \min_{W_r,H_r} \| X - W_rH_r \|_F$. Plot the approximation error as a function of $r$. (Note the subscript $r$ makes the choice of $r$ explicit.) Choose at least 10 different choices of $r$. Based on this plot, what can you say about choosing $r$?

In [ ]:
## Non-negative Matrix Factorization
def non_negative_marix_decomp(n_components,train_data):
    import sklearn.decomposition as skld
    model = skld.NMF(n_components=n_components, init='nndsvda', max_iter=500, random_state=0)
    W = model.fit_transform(train_data)
    H = model.components_
    nmf = (W,H)
    return(nmf)

In [ ]:
w_3,H_3=non_negative_marix_decomp(3,X)
w_7,H_7=non_negative_marix_decomp(7,X)
w_10,H_10=non_negative_marix_decomp(10,X)

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle

import helper_basketball as h
import imp
imp.reload(h);
allshots = pickle.load(open('allshots2016-17.pkl', 'rb'))
allmade = allshots
## bin edge definitions in inches
xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

## 2d histogram containers for binned counts and smoothed binned counts
all_counts = {}
all_smooth = {}

## data matrix: players (row) by vectorized 2-d court locations (column)
for i, one in enumerate(allmade.groupby('PlayerID')):
    
    ## what does this line do?
    pid, pdf = one
    
    ## h.bin_shots: what is this function doing?
    tmp1, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
    tmp2, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
    ## vectorize and store into dictionary
    # see: https://stackoverflow.com/questions/18691084/what-does-1-mean-in-numpy-reshape
    all_smooth[pid] = tmp1.reshape(-1)
    all_counts[pid] = tmp2.reshape(-1)

In [ ]:
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = h.get_nba_data('commonallplayers', params).set_index('PERSON_ID')
## players info
player_ids = allmade.PlayerID.unique()
num_players = player_ids.size
# gives person_id of Kyrie Irving, since person_id is set as the index for players.
players[players.DISPLAY_LAST_COMMA_FIRST == "Irving, Kyrie"].index
player_ids
#gives index of Kyrie Irving 
np.argwhere(player_ids == 202681).flatten()

In [ ]:
#Visualziing R=3 components
fig, ax = plt.subplots(1, 3, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(w_3[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))

In [ ]:
X3_hat = np.matmul(w_3,H_3) 

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,168], xedges, yedges, ax=ax[0]) 
# 168 corresponds to Kyrie Irving 
h.plot_shotchart(X3_hat[:,168], xedges, yedges, ax=ax[1])
ax[0].set_title('Kyrie Irvings Original Shooting Pattern')
ax[1].set_title('Kyrie Irvings Estimated Shooting Pattern (r=3)')

In [ ]:
#Visualziing R=7 components
fig, ax = plt.subplots(1, 7, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(w_7[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))

In [ ]:
X7_hat = np.matmul(w_7,H_7) 

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,168], xedges, yedges, ax=ax[0]) 
# 168 corresponds to Kyrie Irving 
h.plot_shotchart(X7_hat[:,168], xedges, yedges, ax=ax[1])
ax[0].set_title('Kyrie Irvings Original Shooting Pattern')
ax[1].set_title('Kyrie Irvings Estimated Shooting Pattern (r=7)')

In [ ]:
#Visualziing R=10 components
fig, ax = plt.subplots(1, 10, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(w_10[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))

In [ ]:
X10_hat = np.matmul(w_10,H_10) 

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,168], xedges, yedges, ax=ax[0]) 
# 168 corresponds to Kyrie Irving 
h.plot_shotchart(X10_hat[:,168], xedges, yedges, ax=ax[1])
ax[0].set_title('Kyrie Irvings Original Shooting Pattern')
ax[1].set_title('Kyrie Irvings Estimated Shooting Pattern (r=10)')

 From our estimated R's, it appears that R = 7 and R = 10 are too large because it can be seen from the graphs that the basis overlap, therefore we can reduce our R.
Therefore, it appears that R = 3 is the best choice, because there does not exist overlap and each basis represents where players might shoot from realistically. Basketball shots tend to be layups, 3 pointers or 2 pointers. (which lie in the space between layups and 3 pointers) These areas are covered by the R=3 basis. 

## Problem 2

__PSTAT 134 and 234__: In the previous question, NMF gave us a set of bases to describe each player. So, the comparison is through a standard set of shooting styles. We may also approach the comparison more directly.

* In this problem, we compare of players' shooting styles to each other directly. What we are interested in is pairwise correlation between shooting patterns. Let $X_i$ represent the column in the smoothed shooting pattern for player $i$. Then, we want to compute   
    $$ R = [\text{Cor} (X_i, X_j)]_{i,j} $$ for all player combinations $i,j\in\{1,2,\dots,362\}$. What is the correct orientation of matrix $X$? What should be the dimension of matrix $R$?   
    _Note: if your command is not running properly, you may be running into the issue of using too much memory, and your notebook session is rebooted by the server as a result._
    
* Visualize matrix $R$ with [seaborn.heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) function.

* Identify 2 pairs of players with highest similarities (positive correlation) and 2 pairs with lowest similarity (negative correlation). Plot their shooting pattern. What do you observe?

__PSTAT 234 (optional for 134)__: Perform hierarchical clustering with matrix $R$, and visualize the clustered matrix.

NO need to change the orientation of x. It should be row orientes. R is N x N 

In [ ]:
R = np.corrcoef(X,rowvar=0) # Transpose to compute correlation between colums
print(R)

In [ ]:
import seaborn as sns; sns.set()
ax = sns.heatmap(R)

In [ ]:
import numpy as np
# take upper triangular of correlation matrix to get rid of 1's
UpTri=np.triu(R,k=1)
# return the non-zero min and max of the upper diagonal matrix
minval = np.min(UpTri[np.nonzero(UpTri)])
maxval = np.max(UpTri)
# return the index of the min and max correlated players
minval_index = np.where(UpTri == minval)
maxval_index=np.where(UpTri == maxval)
print(minval_index)
print(maxval_index)

In [ ]:
# convert matrix to list in order to return the player ids for the least and most correlated using the above indices
listplayer_ids=player_ids.tolist()
mina=listplayer_ids[23]
minb=listplayer_ids[251]
maxa=listplayer_ids[235]
maxb=listplayer_ids[283]
# now using the players ids to return the name of the most and least correlated, because the players data frame
# uses the player ids as indices
minplayera=players.get_value(mina,"DISPLAY_LAST_COMMA_FIRST")
minplayerb=players.get_value(minb,"DISPLAY_LAST_COMMA_FIRST")
maxplayera=players.get_value(maxa,"DISPLAY_LAST_COMMA_FIRST")
maxplayerb=players.get_value(maxb,"DISPLAY_LAST_COMMA_FIRST")
print(minplayera)
print(minplayerb)
print(maxplayera)
print(maxplayerb)

Shooting Patterns of least correlated players

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,23], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X[:,251], xedges, yedges, ax=ax[1])
ax[0].set_title('Will Barton Original Shooting Pattern')
ax[1].set_title('Mike Muscala Original Shooting Pattern')

Shooting Patterns of Most Correlated Players

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,60))
h.plot_shotchart(X[:,235], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X[:,283], xedges, yedges, ax=ax[1])
ax[0].set_title('Salah Mejri Original Shooting Pattern')
ax[1].set_title('Zach Randolph Original Shooting Pattern')

Here we can see that the least correlated have opposite shooting patterns and the most correlated have almost identical shooting patterns as expected, because a correlation close to 1 would indicate similar bases, while a correlation close to -1 indicates opposing bases.

## Problem 3

__PSTAT 134 and 234__: How would you use the coefficients matrix $H$ from NMF  or the correlation matrix $R$ (computed above) to differentiate between types of players? Consider what the coefficients represent, and how you can use them to discriminate player types.

Give your thought process, reasoning for your chosen method, and the results. Do they look reasonable? Do you expect any of the comparison to be similar to any of the [figures here](https://fastbreakdata.com/classifying-the-modern-nba-player-with-machine-learning-539da03bb824)? Why, or why not? Can you verify your intuition?

## Problem 4

__PSTAT 134 and 234__: Suppose you are in charge of a basketball team. How would you use this information? How would you use what you have learned from analyzing the data, and what other questions would you like to answer with further analysis.

You can use this information to counter attack the offense of the other team. By knowing where a player's "hot spots" are you can coordinate the defense to force the offense to shoot from their least comfortable position. For example you can use the correlation matrix to find the shooting style that is least correlated to a specific player and then use that information to force the offenseive player to shoot from these spots, if the defense is coordinated correctly. (assuming that players that are least correlated signify possible weak points)